# Scraping de la liste des PRADA sur le site de la CADA

http://www.cada.fr/personnes-responsables,6059.html

Car le formulaire n'est pas très convivial...

In [88]:
import re
import sys
import pickle

import requests
from bs4 import BeautifulSoup

# Scrap le formulaire

In [5]:
url = 'http://www.cada.fr/personnes-responsables,6059.html'
response = requests.get(url)
assert response.status_code == 200
assert response.encoding == 'utf-8'
text = response.text

In [9]:
soup = BeautifulSoup(text, 'html.parser')

In [19]:
form_type = soup.find(id="type")
list_type = [option['value'] for option in form_type.find_all('option')]

In [18]:
form_dept = soup.find(id="dept")
list_dept = [option['value'] for option in form_dept.find_all('option')]

# Scrap toutes les combinaisons

In [86]:
dict_prada = {}

for type_admin in list_type:
    dict_prada[type_admin] = {}
    print('Scrapping category "{}"'.format(type_admin))
    
    for dept in list_dept:
        sys.stdout.write('.')
        sys.stdout.flush()
        
        data = {
            'id_article': '6059',
            'page': 'article',
            'formulaire_action': 'prada',
            'formulaire_action_args': 'u2Tfw6pP/715JbIUQ9MMpjPSoXmgRPgQIoIu6+OCRphktQbarUOE29+UgR9cPvRSSzHQJpMQwqaMmBWv+kY+FhObKxr+ec0=',
            'type': type_admin,
            'dept': dept,
            'valide': '',
            'nobot': '',
        }

        response = requests.post(url, data=data)
        assert response.status_code == 200
        assert response.encoding == 'utf-8'
        text = response.text
        
        soup = BeautifulSoup(text, 'html.parser')
        
        chapo = soup.find("p", { "class" : "chapo" }).get_text()
        if chapo == "Recherche infructueuse. Il y n'a pas de résultats pour ces critères de recherche":
            blocs = []
        else:
            nb_results = int(re.match('(\d+) résultats pour vos critères de recherche ', chapo).groups()[0])

            blocs = soup.findAll("div", { "class" : "block-article" })
            if len(blocs) != nb_results:
                if (
                    (type_admin == "Établissement public territorial" and 
                    dept == "" and 
                    len(blocs) == 612 and 
                    nb_results == 613)
                    or
                    (type_admin == "Mairie" and 
                    dept == "" and 
                    len(blocs) == 679 and 
                    nb_results == 680)
                        ):
                    print('Count mismatch (already known)')
                else:
                    raise ValueError('Count mismatch : {}, {}, {} vs {}'.format(type_admin, dept, len(blocs), nb_results))
        
        dict_prada[type_admin][dept] = blocs
        
    print('')
      

Scrapping category ""
...........................................................................................................
Scrapping category "Autorité administrative indépendante"
...........................................................................................................
Scrapping category "Autre organisme privé"
...........................................................................................................
Scrapping category "Conseil départemental"
...........................................................................................................
Scrapping category "Conseil régional"
...........................................................................................................
Scrapping category "Établissement public d'État"
...........................................................................................................
Scrapping category "Établissement public territorial"
.Count mismatch (already known)
.............

# Save the dict

In [95]:
dict_prada_str = {}

for type_admin in list_type:
    dict_prada_str[type_admin] = {}
    for dept in list_dept:
        blocs = dict_prada[type_admin][dept]
        dict_prada_str[type_admin][dept] = [str(prada) for prada in blocs]

In [97]:
with open('dict_prada_str.pickle', 'wb') as f:
    pickle.dump(dict_prada_str, f)

In [99]:
with open('dict_prada_str.pickle', 'rb') as f:
    dict_prada_str = pickle.load(f)

# Output the results

In [127]:
html_beginning = '''
<!DOCTYPE html>
<html lang="fr">
  <head>
    <meta charset="utf-8">
    <title>Liste des PRADA</title>
    <style>
body {
  font: 75%/1.5 Arial, Helvetica, sans-serif;
  color: #535353;
  background: #f2f2f2;
}
h2.titre_type {
    margin-bottom: .5em;
    color: #23aee3;
    font-family: "Trebuchet MS", Verdana, "Lucida Grande", Tahoma, Helvetica, sans-serif;
    font-size: 2.5em;
    line-height: 1.25;
}
h2.titre_dept {
    margin-bottom: .5em;
    color: #23aee3;
    font-family: "Trebuchet MS", Verdana, "Lucida Grande", Tahoma, Helvetica, sans-serif;
    font-weight: normal;
    font-size: 2em;
    line-height: 1.25;
}
h2.h3 {
    font-size: 1.3333333333333333em;
    margin-bottom: .25em;
    color: #244668;
    font-weight: bold;
    font-family: "Trebuchet MS", Verdana, "Lucida Grande", Tahoma, Helvetica, sans-serif;
}
.prada {
    color: #23AEE3;
    font-size: 16px;
    margin-bottom: 0px;
}
    </style>
  </head>
  <body>
'''

html_end = '''
  </body>
</html>
'''

html_page = html_beginning

for type_admin in list_type:
    html_page += '<h2 class="titre_type">{}</h2>'.format(type_admin or 'Toutes catégories')
    for dept in list_dept:
        html_page += '<h2 class="titre_dept">{}</h2>'.format(dept or 'Tous les départements')
        blocs = dict_prada_str[type_admin][dept]
        for bloc in blocs:
            html_page += bloc

html_page += html_end

In [128]:
with open('Liste_PRADA.html', 'w') as f:
    f.write(html_page)